In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

In [1]:
# Классический набор данных для проверки качества работы алгоритмов классификации
from sklearn.datasets import load_iris

In [3]:
sns.set(rc={'figure.figsize': (11.7, 8.27)})

In [4]:
def custom_compare(x, y):
    if str(x) != str(y):
        raise RuntimeError(f'Ожидаемое значение: {y}. Фактическое: {x}')

# Логистическая регрессия

In [12]:
def logistic_regression_predict_solution(w, data, factor_names):
    """
    На основе переданного вектора весов для каждого объекта из
    набора данных делает предсказание с помощью модели логистической регрессии.
    
    Аргументы:
        w: Вектор весов модели логистической регрессии. Первая координата вектора соответствует
           свободному коэффициенту, последующие — весам факторов.
        data: Таблица с объектами, для которых необходимо сделать предсказания.
              Каждый объект описывается набором численных факторов. 
              В данных может быть представлено больше факторов, чем модель использует для предсказания. 
              Искусственного константного фактора, который для всех объектов равен `1` и 
              который используется моделью для предсказания, в таблице нет.
        factor_names: Список названий факторов, которые используются для предсказания. 
                      Порядок названий совпадает с порядком, в котором идут коэффициенты факторов
                      в векторе весов `w`.
        
    Возвращаемое значение:
        Вектор предсказанных вероятностей принадлежности положительному классу для объектов из переданной таблицы.
        Значения в векторе должны быть округлены до 2 знаков после запятой.
    """
    
    ln = len(data[factor_names[0]])
    
    data = data[factor_names].to_numpy()
    
    M = np.ones((ln, 1))
    M = np.hstack([M, data])
    
    M = M.dot(w)
    M = np.exp(-M)

    ans = []
    for i in range(len(M)):
        ans.append(1 / (1 + M[i]))

    ans = np.array(ans)

    return ans.round(2)

In [6]:
def logistic_regression_predict_test():
    w_example_1 = np.array([0.1, 1])
    data_example_1 = pd.DataFrame({
        'x': [0.22, -0.41],
    })
    
    res_example_1 = np.array([0.58, 0.42])
    
    custom_compare(logistic_regression_predict_solution(w_example_1, data_example_1, ['x']), res_example_1)
    
    w_example_2 = np.array([0.1, 2.7, 2.3, -4.1])
    data_example_2 = pd.DataFrame({
        'x': [0.58, 0.15],
        'extra': [1, 2],
        'y': [0.58, 0.19],
        'z': [0.93, 0.44]
    })
    
    res_example_2 = np.array([0.31, 0.3])
    
    custom_compare(logistic_regression_predict_solution(w_example_2, data_example_2, ['x', 'y', 'z']), res_example_2)

    print('Тест прошёл успешно!')

In [13]:
logistic_regression_predict_test()

Тест прошёл успешно!


In [84]:
def logistic_summary_loss_gradient_solution(w, X, y):
    """
    Вычисляет градиент функции суммарных потерь в случае обучения модели логистической регрессии.
    
    Аргументы:
        w: Вектор весов модели логистической регрессии. Первая координата вектора соответствует
           свободному коэффициенту, последующие — весам факторов.
        X: Матрица значений факторов.
           Первый столбец матрицы содержит значения константного коэффициента, который для всех объектов равен 1.
           Колонке с индексом i в матрице соответствует вес с индексом i в векторе w.
        y: Вектор классов, которым в реальности принадлежат объекты из матрицы X.
           Объекту в i-ой строчке матрицы X соответствует значение с индексом i вектора y.
        
    Возвращаемое значение:
        Вектор градиента функции суммарных потерь.
        Каждая координата вектора должна быть округлена до 2 знаков после запятой.
    """
    n = len(y)
    #x_i = X.dot(w)
    ans = []
    for i in range(n):
      loss = -y[i] * (1 / (1 + np.exp(-(-y[i] * X[i].dot(w))))) * X[i]
      ans.append(loss)
      

    return sum(ans)


In [16]:
def logistic_summary_loss_gradient_test():
    w_example_1 = np.array([-0.2, -0.3])
    X_example_1 = np.array([
        [1, -2.1],
        [1, 3.7]
    ])

    y_example_1 = np.array([-1, -1])
    
    res_example_1 = np.array([0.82, -0.49])
    
    custom_compare(logistic_summary_loss_gradient_solution(w_example_1, X_example_1, y_example_1), res_example_1)
    
    w_example_2 = np.array([0.3, -0.1, 0.4])
    X_example_2 = np.array([
        [1, 0.1, 0.3],
        [1, -0.2, 0.5]
    ])

    y_example_2 = np.array([1, -1])
    
    res_example_2 = np.array([0.23, -0.17,  0.19])
    
    custom_compare(logistic_summary_loss_gradient_solution(w_example_2, X_example_2, y_example_2), res_example_2)

    print('Тест прошёл успешно!')

In [85]:
logistic_summary_loss_gradient_test()

Тест прошёл успешно!


In [78]:
def logistic_regression_solve_solution(data, factor_names, y_name, 
                                       learning_rate=0.01, eps=0.1):
    """
    С помощью градиентного спуска строит модель логистической регрессии по переданному набору данных.
    
    Аргументы:
        data: Таблица с объектами обучающей выборки.
              Каждый объект описывается набором численных факторов. 
              В данных может быть представлено больше факторов, чем модель должна использовать для предсказания. 
              Искусственного константного фактора, который для всех объектов равен 1 и 
              который будет использоваться моделью для предсказания, в таблице нет.
        factor_names: Список названий факторов, которые модель должна использовать для предсказания.
        y_name: Название столбца таблицы, в котором для каждого объекта содержится значение предсказываемого класса.
                Класс может иметь значение либо -1, либо 1.
        learning_rate: Опциональный параметр. Коэффициент скорости обучения, который используется в алгоритме градиентного спуска.
        eps: Опциональный параметр. Минимальное расстояние между текущей точкой градиентного спуска и следующей,
             при котором работа алгоритма останавливается.
        
    Возвращаемое значение:
        Возвращает вектор весов модели. 
        Координата вектора с индексом 0 соответствует свободному коэффициенту модели.
        Координата вектора с индексом i соответствует фактору с индексом i - 1 в списке factor_names.
    """
    
    X = data[factor_names].to_numpy()
    y = data[y_name].to_numpy()
    
    ones = np.ones((len(y), 1))
    
    X = np.hstack([ones, X])
    
    # Необходимо задать стартовый набор весов, который является начальной
    # точкой для алгоритма градиентного спуска.
    # В качестве стартового набора весов необходимо использовать вектор, состоящий из значений 0
    w_cur = np.zeros(len(X[0]))
    
    while True:
        # Вычисление градиента с помощью функции logistic_summary_loss_gradient_solution.
        # Важно убрать округление результата работы функции до 2 знаков после запятой
        gradient_value = logistic_summary_loss_gradient_solution(w_cur, X, y)
    
        # Полезно уменьшить значение градиента, разделив каждую его координату на число объектов
        # в выборке, на которой происходит обучение модели
        gradient_value /= len(y)
        
        # Классический шаг градиентного спуска: переход из текущей точки в направлении,
        # противоположном вектору градиента
        w_new = w_cur - (learning_rate * gradient_value)
        
        # Проверка того, что расстояние между текущей точкой и новой не стало меньше или равно eps
        # print(np.linalg.norm(w_cur - w_new))
        if np.linalg.norm(w_cur - w_new) <= eps:
            break
            
        w_cur = w_new
    
    return w_cur.round(2)

In [37]:
def logistic_regression_solve_test():
    data_example_1 = pd.DataFrame({
        'x1': [0.3, -0.1, -0.1, 0.4],
        'x2': [0.1, -0.1, 0.2, -0.1],
        'y': [1, -1, -1, 1],
    })

    factor_names_example_1 = ['x1', 'x2']
    y_name_example_1 = 'y'
    
    res_example_1 = np.array([-0.07, 0.94, -0.1])
    
    custom_compare(logistic_regression_solve_solution(data_example_1, factor_names_example_1, y_name_example_1,
                                                      learning_rate=0.001, eps=0.0001),
                  res_example_1)
    
    
    iris = load_iris()
    
    y_example_2 = (iris.target >= 1).astype('int64').reshape((len(iris.target), 1))
    y_example_2[y_example_2 == 0] = -1
    
    factor_names_example_2 = ['x1', 'x2']
    y_name_example_2 = 'y'
    
    data_example_2 = pd.DataFrame(
        columns=['x1', 'x2', 'x3', 'x4'] + [y_name_example_2],
        data=np.hstack([iris.data, y_example_2])
    )
    
    res_example_2 = np.array([-0.23,  1.17, -1.83])
    
    custom_compare(logistic_regression_solve_solution(data_example_2, factor_names_example_2, y_name_example_2,
                                                      learning_rate=0.01, eps=0.001),
                   res_example_2)
    
    print('Тест прошёл успешно!')

In [38]:
def logistic_regression_solve_res():
    # Загрузка набора данных для тестирования алгоритмов классификации
    iris = load_iris()
    
    # Приведение классов, которые необходимо научиться предсказывать, к значениям -1 и 1
    y = (iris.target > 1).astype('int64').reshape((len(iris.target), 1))
    y[y == 0] = -1
    
    # Создание таблицы на основе набора данных.
    # Факторы, которые есть в данных, будут называться 'x1', 'x2', 'x3' и 'x4'.
    # Классы объектов помещаются в колонку 'y'
    data = pd.DataFrame(
        columns=['x1', 'x2', 'x3', 'x4', 'y'],
        data=np.hstack([iris.data, y])
    )
    
    # Для предсказания будут использоваться только факторы 'x1' и 'x2'
    factor_names = ['x1', 'x2']
    # Предсказываемая характеристика — 'y'
    y_name = 'y'
    
    # Определение оптимальных весов для разработанной модели логистической регрессии
    ws = logistic_regression_solve_solution(data, factor_names, y_name,
                                            learning_rate=0.01, eps=0.001)
    
    for i in range(len(ws)):
        print(f'w{i}:', ws[i])

In [86]:
logistic_regression_solve_test()

0.00011319231422671772
0.00011319231422671772
0.00011319231422671771
0.00011319231422671774
0.00011319231422671774
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.00011319231422671768
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.0001131923142267179
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.00011319231422671746
0.000

In [88]:
logistic_regression_solve_res()

0.009182031365661958
0.008177062471735596
0.00728990464805801
0.006509126481760478
0.00582244141232868
0.0052189709075001894
0.004689679117191519
0.004226910718506153
0.0038216871783139064
0.0034691231040583016
0.0031620728082136958
0.0028962996315221846
0.0026662947240610043
0.0024691459612136684
0.0023000999012120194
0.0021557557066297357
0.002033548294648217
0.0019310677529974637
0.001844606552447795
0.0017729225088035342
0.0017127465921405011
0.0016635285122627488
0.0016224953346962575
0.0015892274852896274
0.0015615738499631933
0.0015389215560110742
0.0015201463379848944
0.0015052705921380187
0.0014922487132586878
0.0014819959514114714
0.001473567402221182
0.001466879530007683
0.0014605449515696406
0.0014559490223066073
0.0014514859818667068
0.0014478941029417002
0.0014446472849030635
0.0014422855319095285
0.0014400506163943598
0.0014374078521189927
0.0014354979004590047
0.0014339772197167807
0.00143239814452702
0.0014308864385408117
0.0014297085950174119
0.0014284252401391736
0.0